Data Cleaning & Preprocessing

In [1]:
import pandas as pd
import numpy as np

# 1. Load the dataset and identify date columns
df = pd.read_csv('layer2.csv')
date_cols = [col for col in df.columns if col not in ['id', 'symbol', 'name', 'platforms']]

# Melt wide format to long format (Token, Date, Price)
df_long = df.melt(id_vars=['symbol', 'name', 'platforms'], value_vars=date_cols, 
                  var_name='Date', value_name='Price')
df_long.dropna(subset=['Price'], inplace=True)            # remove rows with no price data
df_long['Date'] = pd.to_datetime(df_long['Date'], format='%m/%d/%y')  # parse dates

# Clean Platform field (remove newlines for CSV cleanliness)
df_long['platforms'] = df_long['platforms'].str.replace(r'\n\s*', ' ', regex=True)

# Rename and organize columns
df_long.drop(columns=['symbol'], inplace=True)            # drop token symbol (not needed in output)
df_long.rename(columns={'name': 'Token', 'platforms': 'Platform'}, inplace=True)

# Sort by Token and Date (ensuring each token's data is chronological)
# Preserve original token order (as in input file) when sorting
token_order = {name: i for i, name in enumerate(df['name'])}
df_long['TokenOrder'] = df_long['Token'].map(token_order)
df_long.sort_values(['TokenOrder', 'Date'], inplace=True)
df_long.drop(columns=['TokenOrder'], inplace=True)        # drop helper column

# 2. Calculate daily percentage returns per token
df_long['Daily Return'] = df_long.groupby('Token')['Price'].pct_change()

# 3. Create Indexed Price (base = 100 at first date for each token)
df_long['Indexed Price'] = df_long['Price'] / df_long.groupby('Token')['Price'].transform('first') * 100

# 4. Compute per-token statistics: Avg Return, Volatility, Total Return
stats = df_long.groupby('Token').agg(
    **{
        'Avg Return': ('Daily Return', 'mean'), 
        'Volatility': ('Daily Return', 'std'),
        'Total Return': ('Price', lambda x: x.iloc[-1] / x.iloc[0] - 1)
    }
)
stats.fillna(0.0, inplace=True)  # if a token has no variation (e.g., only one price), fill stats with 0
stats.reset_index(inplace=True)

# 5. Merge statistics back into the long dataframe
df_final = pd.merge(df_long, stats, on='Token', how='left')

# Add Volume column (not available in source, so fill with NaN/empty)
df_final['Volume'] = np.nan

# Reorder columns to the desired output order
df_final = df_final[[
    'Date', 'Token', 'Price', 'Volume', 'Daily Return', 
    'Indexed Price', 'Avg Return', 'Volatility', 'Total Return', 'Platform'
]]

# 6. Output to CSV
df_final.to_csv('layer2_tableau_ready.csv', index=False, date_format='%Y-%m-%d')


Generating Token Correlation Data for Heatmap

In [3]:
# 1. Load the processed Layer 2 dataset (the one with daily returns, created previously)
df_final = pd.read_csv('layer2_tableau_ready.csv')

# Expected columns in df_final include:
#   Date, Token, Price, Volume, Daily Return, Indexed Price, 
#   Avg Return, Volatility, Total Return, Platform

# 2. Pivot the table so each Token's daily returns become a separate column
#    Rows = distinct dates, Columns = token names, Values = daily returns
df_pivot = df_final.pivot_table(
    index='Date',         # each row = one date
    columns='Token',      # each column = one token
    values='Daily Return' # the numeric values = daily returns
)

# 3. Compute the correlation matrix
#    This will produce an N x N matrix, where N is the number of tokens
corr_matrix = df_pivot.corr()  # default = Pearson correlation of columns

# 4. Convert the wide correlation matrix into a long format:
#    Columns: ['Token', 'Token2', 'Correlation']
corr_long = corr_matrix.reset_index().melt(
    id_vars='Token',
    var_name='Token2',
    value_name='Correlation'
)

# 5. Save the correlation pairs to a new CSV
corr_long.to_csv('layer2_correlations.csv', index=False)

print("Correlation matrix saved to 'layer2_correlations.csv'.")

Correlation matrix saved to 'layer2_correlations.csv'.
